In [ ]:
import pandas as pd
from datetime import datetime
import requests
import sys
import json
from pandas.io.json import json_normalize
from bs4 import BeautifulSoup
import time
import random
import operator
import re

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
#/Users/leopoldo.alarcon/Documents/scriptsRRSS/chromedriver 2
#https://pbpython.com/pandas-html-table.html

In [ ]:
from webdriver_manager.chrome import ChromeDriverManager

browser = webdriver.Chrome(ChromeDriverManager().install())



====== WebDriver manager ======
Current google-chrome version is 101.0.4951
Get LATEST chromedriver version for 101.0.4951 google-chrome
Driver [/Users/raul.castillo/.wdm/drivers/chromedriver/mac64/101.0.4951.41/chromedriver] found in cache


In [ ]:
#browser = webdriver.Chrome(executable_path=r"/Users/raul.castillo/SCRIPTS/chromedriver 96")

In [ ]:
browser.get('https://www.amazon.com/-/es/Sparkling-Prebiotic-Strawberry-beneficios-intestinal/product-reviews/B084L3Z6LY/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews')
# Previo a correr la liga, hay que verificar que en la página, estamos dentro de los comentarios.

In [ ]:
elem = browser.find_element_by_xpath("//*")

In [ ]:
soup = BeautifulSoup(elem.get_attribute('innerHTML'),'html.parser')

In [ ]:
rev = soup.find_all("div",'a-section review aok-relative')

In [ ]:
len(rev)

10

In [ ]:
#pages = range(86,88,1)
data = pd.DataFrame(columns=['FECHA', 'LUGAR', 'USUARIO', 'RATE', 'TITULO', 'REVIEW', 'GENERO','PROYECTO',
       'PRODUCTO'])

In [ ]:
#APLICACION DE GENERO A COMENTARIOS

def clean_text(txt):
    txt = re.sub("[^a-záéíóúñüäë]", " ", txt.lower())
    # Nos pasa el textoo a minúsculas.
    txt = re.sub(' +',' ', txt)
    # Limpia nuestro texto.
    return txt.strip().split()
    # Nos regresa el texto con las delimitaciones especificadas.

def df_to_dict(df, key_column, val_column):
    """convierte dos pandas series en un diccionario"""
    xkey = df[key_column].tolist()
    xval = df[val_column].tolist()
    return dict(zip(xkey,xval))

def get_gender2(names):
    names = clean_text(names)
    names = [x for x in names if gender_list.get(x,'a') != 'a']
    gender ={'m':0, 'f':0, 'a':0}
    for i, name in enumerate(names):
        g = gender_list.get(name,'a')
        gender[g] += 1
        gender[g] += 2 if len(names) > 1 and i == 0 and g != 'a' else 0 
    gender['a'] = 0 if (gender['f']+gender['m']) > 0 else 1
    return max(gender.items(), key=operator.itemgetter(1))[0]

gender_list = pd.read_csv('nombres.csv')
gender_list = df_to_dict(gender_list, key_column='nombre', val_column='genero')

In [ ]:
pages = range(1,10,1)
#inicio de rango, hasta dónde llega el rango (el fin) es el número de páginas, saltos (1 en 1, 3 en 3 o 20 en 20)
# Va a correr 160 veces (Conciderando que tenemos 1600 comentarios y 10 comentarios por página, multiplicamos 10x160 = 1600)
url1='https://www.amazon.com/-/es/Sparkling-Prebiotic-Strawberry-beneficios-intestinal/product-reviews/B084L3Z6LY/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews'
# Previo a correr la liga, hay que verificar que en la página, estamos dentro de los comentarios.
url2='&pageNumber='
producto = url1[len('https://www.amazon.com.mx/'):len('https://www.amazon.com.mx/')+url1[len('https://www.amazon.com.mx/'):].find('/')].replace('+',' ')
browser.maximize_window()
for p in pages:
    desplazar = random.randint(400, 1200)
    browser.execute_script('window.scrollTo(1, '+str(desplazar)+');')
    time.sleep(2)
    urlr = url1+str(p)+url2+str(p)
    print(urlr)
    browser.get(urlr)
    elem = browser.find_element_by_xpath("//*")
    soup = BeautifulSoup(elem.get_attribute('innerHTML'),'html.parser')
    reviews = soup.find_all("div",'a-section review aok-relative')
    print(len(reviews),len(data),desplazar)
    for rev in reviews:
        #print(com.get_text(),'\n')
        fecha = rev.find_all("span",'a-size-base a-color-secondary review-date')[0].get_text()
        fechas = fecha.split('el',1)[1].strip(' ')
        lugar = fecha.split('el',1)[0].split('en',2)[1].strip(' ')
        autor = rev.find_all("span",'a-profile-name')[0].get_text()
        gen = get_gender2(autor)
        rate = rev.find_all("span",'a-icon-alt')[0].get_text()
        rate = rate.split('de',1)[0]
        #title = rev.find_all("",'a-size-base a-link-normal review-title a-color-base review-title-content a-text-bold')[0].find_all("span",'')[0].get_text()
        title = rev.find_all("a",'review-title')[0].text.strip('\n')
        #comen = rev.find_all("span",'a-size-base review-text review-text-content')[0].find_all("span",'')[0].get_text()
        #comen = comen.strip('\n')
        comen = rev.find_all("span",'a-size-base review-text review-text-content')[0].text.strip('\n').strip(' ')
        #comentario = pd.DataFrame([[fechas,lugar,autor,rate,title,comen,gen]])
        #data = data.append(comentario,ignore_index=True)
        data = data.append({'FECHA': fechas,'LUGAR':lugar,'USUARIO':autor,'RATE':rate,'TITULO':title,
                    'REVIEW':comen,'GENERO':gen,'PROYECTO':'SINZU','PRODUCTO':producto},ignore_index=True)
    sleep = random.randint(19, 29) # Hará el scroll entre 19 y 29 segundos
    #print(sleep)
    time.sleep(sleep)

https://www.amazon.com/-/es/Sparkling-Prebiotic-Strawberry-beneficios-intestinal/product-reviews/B084L3Z6LY/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews1&pageNumber=1
10 0 553
https://www.amazon.com/-/es/Sparkling-Prebiotic-Strawberry-beneficios-intestinal/product-reviews/B084L3Z6LY/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews2&pageNumber=2
10 10 505
https://www.amazon.com/-/es/Sparkling-Prebiotic-Strawberry-beneficios-intestinal/product-reviews/B084L3Z6LY/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews3&pageNumber=3
10 20 917
https://www.amazon.com/-/es/Sparkling-Prebiotic-Strawberry-beneficios-intestinal/product-reviews/B084L3Z6LY/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews4&pageNumber=4
10 30 950
https://www.amazon.com/-/es/Sparkling-Prebiotic-Strawberry-beneficios-intestinal/product-reviews/B084L3Z6LY/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews5&pageNumber=5
10 40 403
https://www.amazon.com/-/es/Sparklin

In [ ]:
data

,FECHA,LUGAR,USUARIO,RATE,TITULO,REVIEW,GENERO,PROYECTO,PRODUCTO
0,7 de abril de 2020,Estados Unidos,Amazon Customer,2.0,Made with Stevia,"Upon seeing this product on Shark Tank, we wer...",a,SINZU,s
1,29 de marzo de 2020,Estados Unidos,Samer K.,5.0,Takes you to flavor town! With Only 15cal.,Talk about flavor!The Orange flavor is my favo...,m,SINZU,s
2,23 de marzo de 2021,Estados Unidos,Cary Murray,5.0,Great alternative to wine when trying to cut b...,I got turned on to Poppi in December 2020 afte...,m,SINZU,s
3,30 de mayo de 2021,Estados Unidos,Quackers719,5.0,Great alternative to regular sodas!,I am obsessed with Poppi. I would invest in th...,a,SINZU,s
4,3 de mayo de 2020,Estados Unidos,Opinionated,3.0,"Too much Stevia, not enough ACV flavor","This beverage has a nice, light, refreshing fl...",a,SINZU,s
...,...,...,...,...,...,...,...,...,...
85,17 de junio de 2020,Estados Unidos,Hannalore,2.0,POPPI is not very tasty,The drink is ridiculously over priced. Not at ...,a,SINZU,s
86,7 de septiembre de 2020,Estados Unidos,Natalie Daniels,1.0,didn't care for the flavors,Didn't care for the flavor and too expensive.,f,SINZU,s
87,30 de junio de 2021,Estados Unidos,Cord and Colleen Gallegos,4.0,"So amazing, but so expensive",I love these so much. Much healthier than soda...,m,SINZU,s
88,11 de septiembre de 2021,Estados Unidos,HidefKd,5.0,My grandson loves these!,We bought these at Whole Foods as an after sch...,a,SINZU,s


In [ ]:
data.groupby(['RATE']).count()

,FECHA,LUGAR,USUARIO,TITULO,REVIEW,GENERO,PROYECTO,PRODUCTO
RATE,,,,,,,,
1.0,11,11,11,11,11,11,11,11
2.0,4,4,4,4,4,4,4,4
3.0,3,3,3,3,3,3,3,3
4.0,11,11,11,11,11,11,11,11
5.0,61,61,61,61,61,61,61,61


In [ ]:
export_csv = data.to_csv ("Sinzu8.csv", index = None, header=True , encoding='UTF-8')

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=61de9947-6a6b-4aa1-8e4b-fbaeeeaf203b' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>